In [295]:
import pandas as pd
import datetime

from os import listdir
from os.path import isfile, join

In [296]:
def get_all_files_to_use(folder_path):
    onlyfiles = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    files_to_use = list(filter(lambda f: 'Retention' in f and 'csv' in f, onlyfiles))
    files_to_use = [folder_path + f for f in files_to_use]
    
    return files_to_use


def data_to_df(files_to_use):
    # Read all data to dataframe
    list_ = []
    for file_ in files_to_use:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)

    data = pd.concat(list_, axis = 0, ignore_index = True)
    
    return data


def create_id(data, columns_to_use):
    # Create new ids for users from existing fields.
    # Order in columns_to_use metters (first columns have priority).
    if len(columns_to_use) > 1:
        first_col = columns_to_use[0]
        data['ID'] = data.apply(lambda row: row[first_col] if row[first_col]!='undefined' else row[columns_to_use[1]],
                                axis=1)
        if len(columns_to_use) > 2:
            for col in columns_to_use[2:]:
                data['ID'] = data.apply(lambda row: row['ID'] if row['ID']!='undefined' else row[col], axis=1)
        return data
    
    else:
        raise ValueError('No columns to use. (There should be > 1 columns in column_to_use)')

        
def get_ids_and_install_date_pairs(data):
    # Returns dict {id: install date}
    installs = data[data['Event']=='install_']
    installs = installs.sort_values(by=['Date'], ascending=True)
    installs = installs.groupby(['ID'])['Date'].first()
    installs = installs.to_dict()
    try:
        installs.pop('nannan')
    except:
        print('no nannan')
    
    return installs


def days_between(d1, d2):
    d1 = datetime.datetime.strptime(d1, "%Y%m%d")
    d2 = datetime.datetime.strptime(d2, "%Y%m%d")
    return abs((d2 - d1).days)


def filter_rows_by_period(data, period_from_install):
    period = {'1 week':[-1,7], '2 week': [7,14], '3 week': [14,21], '4 week': [21,28], 
              '1 month':[0,30], '2 month':[30,60], '3 month':[60,90]}
    data['to_stay'] = data.apply(lambda row: 1 if days_between(row['date_of_install'], row['Date']) 
                                 <= period[period_from_install][1] and 
                                 days_between(row['date_of_install'], row['Date']) > period[period_from_install][0]
                                 else 0, axis=1)
    data = data[data['to_stay']==1]
    return data

In [297]:
def calculate_retention(folder_path, period_from_install='1 week', times_opened=1):
    # Get all file names to use
    files_to_use = get_all_files_to_use(folder_path)
    
    # Read all data to dataframe
    data = data_to_df(files_to_use)
    
    # Leave only necessary columns
    data = data[['Date', 'Device Id', 'Ios Devices', 'Android Devices','Email', 'Event']]
    
    # Transform event names and date format
    data['Event'] = data['Event'].apply(lambda s: s.split(' - Unique')[0])
    data['Date'] = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%b %d \'%y')
                                                  .strftime('%Y%m%d'))
    
    # Create new id based on list of columns
    columns_to_id = ['Email', 'Device Id', 'Ios Devices', 'Android Devices']
    data = create_id(data, columns_to_id)
    
    # Define install as "install" or "First App Open" event
    install = ['install', 'First App Open']
    data['Event'] = data['Event'].apply(lambda event: 'install_' if event in install else event)
    
    # Remove unnecessary columns and duplicated rows
    data = data[['ID', 'Date', 'Event']]
    data = data.drop_duplicates(subset=data.columns, keep='first')
    
    # Get install dates
    installs = get_ids_and_install_date_pairs(data)
    data['date_of_install'] = data.apply(lambda row: installs[row['ID']] 
                                         if row['ID'] in installs.keys()
                                         else 'unknown', axis=1)
    data = data[data['date_of_install']!= 'unknown']
    
    # Leave rows from needeed time period
    data = filter_rows_by_period(data, period_from_install)
    
    # Count events ("App session") for each ID
    data = data[data['Event'] =='App Session']
    data = data.groupby(['ID', 'date_of_install'])['Event'].apply(list).reset_index()
    data['events_count'] = data['Event'].apply(lambda x: len(x))
    
    # Leave only users, that opened app >= times_opened
    data = data[data['events_count'] >= times_opened]
    
    # Count users by date
    result = data.groupby(['date_of_install'])['ID'].apply(list).reset_index()
    result['ID'] = result['ID'].apply(lambda x: len(list(set(x))))
    
    return result

In [293]:
folder_path = '/home/skein/kidadl/Retention Bars/'
result = calculate_retention(folder_path=folder_path, period_from_install='1 week', times_opened=2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [300]:
to_calc = ['1 month', '2 month', '3 week', '1 week', '2 week', '4 week', '3 month']
folder_path = '/home/skein/kidadl/Retention Bars/'
for period in to_calc:
    result = calculate_retention(folder_path=folder_path, period_from_install=period, times_opened=1)
    result.to_csv('Retention transformed/Opens app 1+ times in %s.csv' %period, index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


no nannan
no nannan
no nannan
no nannan
no nannan
no nannan
no nannan


In [298]:
t = {'1 week':[-1,7], '2 week': [7,14], '3 week': [14,21], '4 week': [21,28], 
              '1 month':[0,30], '2 month':[30,60], '3 month':[60,90]}
t.keys()

dict_keys(['1 month', '2 month', '3 week', '1 week', '2 week', '4 week', '3 month'])

In [ ]:
result.to_csv('Opens app 1+ times in 1 week.csv', index=False)
#result.to_csv('Opens app 1+ times in 2 week.csv', index=False)
#result.to_csv('Opens app 1+ times in 3 week.csv', index=False)
#result.to_csv('Opens app 1+ times in 4 week.csv', index=False)
#result.to_csv('Opens app 1+ times in 1 month.csv', index=False)
#result.to_csv('Opens app 1+ times in 2 month.csv', index=False)
#result.to_csv('Opens app 1+ times in 3 month.csv', index=False)